## Implement DH with negotiated groups, and break with malicious "g" parameters

```
A->B
    Send "p", "g"
B->A
    Send ACK
A->B
    Send "A"
B->A
    Send "B"
A->B
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), msg) + iv
B->A
    Send AES-CBC(SHA1(s)[0:16], iv=random(16), A's msg) + iv 
```

 Do the MITM attack again, but play with "g". What happens with:

```
    g = 1
    g = p
    g = p - 1
```

Write attacks for each. 

Alice randomly selects private value $a$ and calculates public value $A = g^{a}\mod{p}$, which is sent to Bob.

For $g = 1$, Alice sends $A = 1^a\mod{p} = 1$. Bob calculates shared secret $s = A^b\mod{p} = 1^b\mod{p} = 1$. In other words, if MITM attacker can inject a malicious $g = 1$ parameter into the DH group negotiation, the shared secret is forced to have value $1$.

For $g = p$ the calculation of shared secret $s$ from Bob's perspective is $s = A^b\mod{p} = (p^a)^b\mod{p} = p^{ab}\mod{p} = 0$, so the shared secret is forced to the value of $0$.

### When does this ever happen?
Honestly, not that often in real-world systems. If you can mess with "g", chances are you can mess with something worse. Most systems pre-agree on a static DH group. But the same construction exists in Elliptic Curve Diffie-Hellman, and this becomes more relevant there.
